In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [14]:
import glob
import os
# reading in dicom files
import pydicom as dicom
from os import listdir
from os.path import isfile, join
import pandas as pd
from collections import Counter

import time


path = '/raid/data01/deephealth/'

folder = 'dh_dcm_ast'
labelPath = os.path.join('/home2/deephealth/labels', folder+'_labels.csv')
df = pd.read_csv(labelPath)

folderPath = str(path) + '/' + str(folder)

files_2D = sorted([f for f in glob.glob(str(folderPath)+ '/*/DXm*')])
files_3D = sorted([f for f in glob.glob(str(folderPath)+ '/*/BT*')])
label_df = pd.DataFrame(columns=['folder', 'Total_dicoms', 'Total_dicoms2D', 'Total_dicoms3D', 'Total_lables2D', 'Total_lables3D'])

def getLabel(studyInstanceUID, imageLaterality, df):
    
    value = df.loc[(df['StudyInstanceUID'] == str(studyInstanceUID))]
    
    if value.shape[0] == 1:
        if value['CancerLaterality'].values[0] == str(imageLaterality):
            return(str(value['Label'].values[0]))
        else:
            return("NOT" + str(value['Label'].values[0]) ) 

    elif value.shape[0] == 0:
        return("None")
    else:
        print("error")
        
        
        
    
    
def getTag(ds, tagValue):        
    tag = ds.get(str(tagValue))
    return tag






def _cal_2DLabel( df, executionTime=False):
    df = df.drop_duplicates ()
    t0 = time.time()
    
    labels = []
    
    i =0 

    for eachFile in files_2D:
        ds = dicom.dcmread(eachFile)
        
        print(i)
        print(eachFile)
        value  = getLabel(getTag( ds, "StudyInstanceUID"), getTag(ds, "ImageLaterality"), df)
        labels.append(value)
        i+=1

    if executionTime:
        print("...took ", time.time()-t0, "seconds")
    print("Total 2D dicoms", len(files_2D))
    print("Total labels", len(labels))
    print(Counter(labels))

    return labels


def _cal_3DLabel( df, executionTime=False):
    t0 = time.time()
    i = 0
    labels = []
    
    df = df.drop_duplicates()
    df = df.sort_values(by='StudyInstanceUID')
    
    for eachFile in files_3D:
        print(i)
        print(eachFile)
        ds = dicom.dcmread(eachFile)
        studyInstanceUID = getTag( ds, "StudyInstanceUID")
        imageLaterality = getTag(ds, "ImageLaterality")
         

        value  = getLabel(studyInstanceUID, imageLaterality , df)
        labels.append(value)
        i = i + 1

    if executionTime:
        print("...took ", time.time()-t0, "seconds")
    print("Total 3D dicoms", len(files_3D))
    print("Total labels", len(labels))
    print(Counter(labels))

    return labels


def createRecord(df, folder, label_df, executionTime=False):
    
    df = df.drop_duplicates()
    df = df.sort_values(by='StudyInstanceUID')
    
    t0 = time.time()
    folder = str(folder)
    
    

    labels_2D = _cal_2DLabel( df, executionTime = True)
    
    labels_3D = _cal_3DLabel( df, executionTime = True)
    
    
    
    
    label_df = label_df.append({'folder': folder,
                    'Total_dicoms': len(files_2D) + len(files_3D),
                    'Total_dicoms2D': len(files_2D),                    
                    'Total_dicoms3D': len(files_3D),
                    'Total_lables2D': labels_2D, 
                    'Total_lables3D': labels_3D,                   
                   }
                   ,ignore_index=True
                  )
    if executionTime:
        print("...took ", time.time()-t0, "seconds")
    
    return label_df







In [12]:
createRecord(df, folder, label_df, executionTime=True)

KeyboardInterrupt: 